Classification means that dependent on the input it might be 1, 2 or 3 as result and the solution is given as percentage factor.

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras as ks
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('absl').setLevel(logging.ERROR)

In [ ]:
model = ks.Sequential()
model.add(ks.layers.Input(shape=(224, 224, 3)))
model.add(ks.layers.Dense(64, activation='relu'))
model.add(ks.layers.Dense(3, activation='softmax'))

model.compile(optimizer=ks.optimizers.Adam(), loss=ks.losses.CategoricalCrossentropy(), metrics=['accuracy'])